In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from collections import Counter

In [2]:
# loading the pre-processed data
processed_data = pd.read_json("processed_data.json")

In [3]:
# randomizing the data
data = processed_data.iloc[np.random.permutation(len(processed_data))]
print(data.shape)

(71410, 3)


In [4]:
#spliting the data into 80:20 training testing ratio

msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]
print(train.shape)
print(test.shape)

(57139, 3)
(14271, 3)


In [14]:
train.columns

Index([u'content', u'id', u'label'], dtype='object')

In [6]:
label_counts =train['label'].value_counts()
label_counts

en         20471
ja         11963
es          6430
und         3352
id          3325
pt          3318
ar          2467
ru          1064
fr          1045
tr           750
ko           523
th           515
tl           405
it           342
de           202
nl           190
ms           142
pl           116
sv            65
el            38
he            37
sr            28
zh-CN         27
fi            22
lv            21
fa            19
uk            18
vi            18
ca            16
jv            15
hi-Latn       14
hi            13
ar_LATN       13
hu            12
ro            12
zh-TW         11
da            11
ur_LATN       11
su            10
sw             9
no             9
sq             8
ur             8
ta             7
ne             6
cs             4
ht             4
hr             4
bg             4
bs             4
ko_LATN        3
az             3
ja_LATN        3
mr             3
km             3
gl             2
xh             2
mn             2
Name: label, d

In [9]:
label_counts_index = label_counts.index
label_counts_index

Index([u'en', u'ja', u'es', u'und', u'id', u'pt', u'ar', u'ru', u'fr', u'tr',
       u'ko', u'th', u'tl', u'it', u'de', u'nl', u'ms', u'pl', u'sv', u'el',
       u'he', u'sr', u'zh-CN', u'fi', u'lv', u'fa', u'uk', u'vi', u'ca', u'jv',
       u'hi-Latn', u'hi', u'ar_LATN', u'hu', u'ro', u'zh-TW', u'da',
       u'ur_LATN', u'su', u'sw', u'no', u'sq', u'ur', u'ta', u'ne', u'cs',
       u'ht', u'hr', u'bg', u'bs', u'ko_LATN', u'az', u'ja_LATN', u'mr', u'km',
       u'gl', u'xh', u'mn'],
      dtype='object')

In [10]:
len(label_counts_index)

58

In [11]:
pp = {}
for i in label_counts_index:
    pp[i] = train.loc[train['label'] == i]
    
print(len(pp)) 
pp['en']

58


,content,id,label
20126,A Like anything donut texture is mood depende...,491317796538945536,en
8140,I still miss you,485874875798941696,en
28090,though i am a bit surprised it said honor roll...,494946736381493248,en
23071,Luke Hemmings I love you more than this ️,492571251353743360,en
21271,Wifi acting mad retarded,491806440256847872,en
26716,I love reading w my bae and Youshithank you fo...,494286071031554048,en
15952,Bloodlust,489387882226782208,en
21974,Lil bitch boy,492097973707624448,en
14895,They both called me,488881152501882880,en
14695,I cant quite believe I just watched a minute ...,488802486090993664,en


In [207]:
def do_vectorize(data,min_df = 1,max_df = 1.0,binary =True,ngram_range = (1,1)):
    
    vec = CountVectorizer(max_df=max_df, min_df=min_df,binary=binary, ngram_range=ngram_range)
    X = vec.fit_transform(data)
    return (X, vec)

In [225]:
cc = {}
most_frequent = {}
for i in pp.keys():
    X,vec = do_vectorize(pp[i]['content'])
    cc[i] = zip(vec.get_feature_names(),np.asarray(X.sum(axis=0)).ravel())
    most_frequent[i] = sorted(cc[i], key=lambda tup: tup[1], reverse=True)[:100]

In [226]:
len(most_frequent)

58

In [227]:
most_frequent['en']

[(u'the', 3797),
 (u'to', 3470),
 (u'you', 3088),
 (u'and', 2339),
 (u'me', 2048),
 (u'my', 2019),
 (u'in', 1917),
 (u'for', 1850),
 (u'is', 1823),
 (u'of', 1711),
 (u'it', 1432),
 (u'on', 1409),
 (u'so', 1273),
 (u'im', 1200),
 (u'this', 1171),
 (u'that', 1130),
 (u'with', 1059),
 (u'be', 1027),
 (u'love', 955),
 (u'just', 910),
 (u'your', 909),
 (u'at', 908),
 (u'have', 842),
 (u'are', 764),
 (u'like', 762),
 (u'not', 748),
 (u'but', 747),
 (u'follow', 746),
 (u'its', 717),
 (u'one', 695),
 (u'if', 671),
 (u'up', 670),
 (u'dont', 667),
 (u'out', 648),
 (u'do', 628),
 (u'from', 614),
 (u'can', 612),
 (u'please', 600),
 (u'what', 596),
 (u'all', 582),
 (u'get', 573),
 (u'new', 573),
 (u'by', 566),
 (u'no', 564),
 (u'was', 560),
 (u'we', 527),
 (u'now', 510),
 (u'amp', 508),
 (u'go', 506),
 (u'when', 498),
 (u'today', 489),
 (u'know', 467),
 (u'more', 466),
 (u'lol', 438),
 (u'how', 428),
 (u'good', 426),
 (u'via', 424),
 (u'day', 412),
 (u'much', 412),
 (u'will', 408),
 (u'about', 406)

In [228]:
def fun(string):
    string_label = []
    for word in string.split():
        #print word

        for sym in most_frequent:
            #print sym

            for i in most_frequent[sym]:

                if i[0] == word:
                    string_label.append(sym)
    if len(string_label) == 0:
        string_label.append('en')
                
    
    c = Counter(string_label)
    #print c
    v=list(c.values())
    k=list(c.keys())
    return k[v.index(max(v))]
                

In [229]:
predicted_labels = []
for i in test['content']:
    #print i
    predicted_labels.append(fun(i))

In [230]:
Counter(string_label)

Counter({u'en': 4, u'mr': 1, u'nl': 1})

In [231]:
predicted_labels

['en',
 u'en',
 u'es',
 u'en',
 u'ar',
 u'fr',
 u'en',
 u'en',
 'en',
 u'lv',
 u'es',
 u'und',
 'en',
 'en',
 u'pt',
 'en',
 'en',
 u'en',
 u'en',
 u'en',
 u'su',
 u'en',
 'en',
 u'en',
 'en',
 'en',
 u'und',
 u'en',
 'en',
 'en',
 'en',
 u'fr',
 'en',
 u'it',
 u'fr',
 'en',
 'en',
 u'en',
 u'en',
 'en',
 'en',
 'en',
 u'en',
 u'pt',
 u'en',
 u'fr',
 u'en',
 'en',
 u'ca',
 u'en',
 u'en',
 u'en',
 u'en',
 'en',
 u'fr',
 u'en',
 u'es',
 'en',
 'en',
 'en',
 u'fr',
 u'el',
 u'ar',
 'en',
 'en',
 'en',
 'en',
 u'fr',
 u'ar',
 u'es',
 u'en',
 u'tl',
 u'es',
 u'en',
 'en',
 'en',
 'en',
 'en',
 u'en',
 'en',
 u'es',
 u'en',
 u'en',
 u'pt',
 'en',
 'en',
 u'en',
 u'tl',
 'en',
 u'es',
 u'fr',
 u'en',
 'en',
 u'en',
 u'fr',
 u'ur_LATN',
 u'en',
 u'es',
 'en',
 'en',
 'en',
 u'en',
 u'en',
 u'en',
 'en',
 u'fr',
 'en',
 'en',
 u'en',
 'en',
 u'en',
 u'en',
 u'en',
 u'en',
 u'und',
 'en',
 u'en',
 'en',
 u'ar',
 u'en',
 'en',
 u'en',
 u'el',
 u'en',
 'en',
 'en',
 u'ar',
 'en',
 u'fr',
 u'id',
 

In [232]:
from sklearn.metrics import accuracy_score
y_pred = test['label']
y_true = predicted_labels
accuracy_score(y_true, y_pred)


0.54989744677841434